In [4]:
from tools.data import get_datasets
#train, test = get_datasets()
mtrain, mtest = remove_class(train, -1),remove_class(test, -1)
btrain, btest = replace_classes(train, [i for i in range(10)],1),replace_classes(test, [i for i in range(10)],1)

#train et test -> Ensemble de données contenant toutes les classes d'exemples (incluant -1)
#mtrain et mtest -> Ensemble de données avec juste des signaux valides (aucune classe -1)
#btrain et btest -> Ensemble donn/es avec classe -1 (pas un signal) et 1 (est un signal)



In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.neighbors import KNeighborsClassifier
from sklearn import naive_bayes
import features


In [ ]:
# Pris du classifieur de Marc

In [5]:
# Fonction permettant d'extraire la puissance des signaux issus du filtre du Butterworth

def butterworth_filter(dataset):
    Wn = [('lowpass', 0.035) , ('bandpass', (0.035,0.07)) , ('bandpass', (0.07, 0.113)), ('bandpass', (0.113, 0.26)), ('highpass', 0.26)]
    butter = np.zeros((dataset.shape[0], 20))
    for i in range(dataset.shape[0]) :
        length = dataset[i,0]
        current = []
        for filter_type, value in Wn :
            b, a = signal.butter(5, value, filter_type)
            current.append((np.sum(signal.filtfilt(b, a, dataset[i, 1:1+length]))**2)/length)
            current.append((np.sum(signal.filtfilt(b, a, dataset[i, 1+512:1+512+length]))**2)/length)
            current.append((np.sum(signal.filtfilt(b, a, dataset[i, 1+512*2:1+512*2+length]))**2)/length)
            current.append((np.sum(signal.filtfilt(b, a, dataset[i, 1+512*3:1+512*3+length]))**2)/length)
        butter[i] = np.array(current)
    return butter

#Pris à partir du classifieur PCA - KNN de Thomas. J'ai pas eu trop le temps de regarder.


In [22]:
from sklearn.decomposition import PCA
# scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA
        
test_f = np.zeros(test.shape)

for i in range(len(test_f)):
    length = test[i,0]
    test_f[i,0] = test[i,0]
    test_f[i,-1] = test[i,-1]
    
    for j in range(4):
        test_f[i, 1+512*j:1+512*j+length] = np.abs(np.fft.fft(test[i, 1+512*j:1+512*j+length]))

def getPCA(n,data):
    train_f = np.zeros(data.shape)

    for i in range(len(train_f)):
        length = data[i,0]
        train_f[i,0] = data[i,0]
        train_f[i,-1] = data[i,-1]

        for j in range(4):
            train_f[i, 1+512*j:1+512*j+length] = np.abs(np.fft.fft(data[i, 1+512*j:1+512*j+length]))

    
    pca = PCA(n)
    return pca.fit_transform(train_f[:,1:200])


/home/ntak/.local/lib/python2.7/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [7]:
def most_resonnant_by_signal(data,nbcapteurs=4,buffer_size=512, numcep=30):
    """
    Returns a matrix of shape ( nb of examples x nb of signals)
    where each element at index i,j is the index of the most resonnant ceptrum by signal.
    """

    feat = np.zeros((data.shape[0], nbcapteurs))
    
    for x, d in enumerate(data): 
        length = int(d[0])
        
        for i in range(nbcapteurs):

            signal = d[1+i*buffer_size:1+i*buffer_size + length]# I do not get a fix number of frame if I do not use the zero-padding.
            feat[x,i] = np.argmax(features.mfcc(signal, buffer_size, numcep=numcep))
    
    return feat

In [8]:
def results(data, nbcapteurs=4, buffer_size=512, numcep=13, numframes=94):
    """ data : longueur | 512 mesures pour capteur 1 | 512 mesures pour capteur 2 | 512 mesures pour capteur 3 | 512 mesures pour capteur 4 | Chiffre vu | """
    return data[:,-1]

def remove_class(data, target, copy=True):
    """
    Remove all examples which class == target.
    assumes that the class is at the last index of the data
    """
    if copy :
        return np.array(data[data[:,-1] != -1],copy=True)
    else:
        return data[data[:,-1] != -1]

def replace_classes(data, to_replace, replace_by, copy = True):
    """
    For all examples which class in to_replace. Chance class for replace_by
    assumes that the class is at the last index of the data
    """
    if copy:
        data = np.array(data,copy=True)
    
    for x in data:
        if x[-1] in to_replace:
            x[-1] = replace_by
    
    return data


#La section de test

La section suivante sert à tester un vecteur de caractéristiques sur plusieurs modèles d'apprentissage ainsi que de trouver leurs hyper-paramètres optimaux (si applicable).

In [ ]:
#initialising feature vector
mfcc = most_resonnant_by_signal(mtrain)
butter = butterworth_filter(mtrain)
pca = getPCA(5,mtrain)
train_result = mtrain[:,-1]




In [32]:
mfcc2= most_resonnant_by_signal(mtest)
butter2= butterworth_filter(mtest)
pca2 = getPCA(5,mtest)
test_result = mtest[:,-1]

/home/ntak/.local/lib/python2.7/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ntak/.local/lib/python2.7/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ntak/.local/lib/python2.7/site-packages/ipykernel/__main__.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ntak/.local/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ntak/.local/lib/python2.7/site-packages/ipykernel/__main__.py:23: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [35]:
train_feat = []
test_feat = []
train_result = mtrain[:,-1]
for i in range(mtrain.shape[0]) :
    #print(pca[i].shape, mfcc[i].shape)
    train_feat.append(np.concatenate((pca[i],mfcc[i],butter[i]))) 


#train_result = mtrain[,:-1]

for i in range(mtest.shape[0]) :
    test_feat.append(np.concatenate((pca2[i],mfcc2[i],butter2[i]))) 

#test_result = mtest[:-1]



In [29]:
print mfcc2

[[ 1799.  1983.   397.  1642.]
 [ 2371.    85.  1671.   423.]
 [ 1801.   683.  1307.  1955.]
 ..., 
 [   59.   163.  1957.  1515.]
 [  683.  1100.   215.   810.]
 [ 1151.    82.  1123.  1255.]]


In [36]:
# On entraine un k-ppv pour différentes valeurs de k

for n in range(10, 100):
    # train set
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(train_feat, train_result)
    
    
    # test set
    predictions = neigh.predict(test_feat)
    total_correct = np.sum(predictions == test_result)

    print 'Pourcentage correct %d:' % n, float(total_correct) / len(test_feat) * 100

Pourcentage correct 10: 9.77275549485
Pourcentage correct 11: 9.98385694772
Pourcentage correct 12: 9.61132497206
Pourcentage correct 13: 9.84726188998
Pourcentage correct 14: 9.69824909971
Pourcentage correct 15: 9.84726188998
Pourcentage correct 16: 9.82242642493
Pourcentage correct 17: 10.0211101453
Pourcentage correct 18: 9.84726188998
Pourcentage correct 19: 9.98385694772
Pourcentage correct 20: 9.99627468024
Pourcentage correct 21: 10.2322115982
Pourcentage correct 22: 10.0583633429
Pourcentage correct 23: 9.90935055259
Pourcentage correct 24: 9.89693282007
Pourcentage correct 25: 9.82242642493
Pourcentage correct 26: 9.92176828511
Pourcentage correct 27: 10.0707810754
Pourcentage correct 28: 9.93418601763
Pourcentage correct 29: 9.90935055259
Pourcentage correct 30: 9.98385694772
Pourcentage correct 31: 9.99627468024
Pourcentage correct 32: 10.1452874705
Pourcentage correct 33: 10.1452874705
Pourcentage correct 34: 10.157705203
Pourcentage correct 35: 10.132869738
Pourcentage co

In [38]:
#Naif Bayes
nbayes = naive_bayes.GaussianNB()

#train
nbayes.fit(train_feat,train_result)

#test
y = nbayes.predict(test_feat)
total_correct = np.sum(predictions == test_result)

print 'Pourcentage correct %d:' % n, float(total_correct) / len(test_feat) * 100

Pourcentage correct 99: 9.53681857693


In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.cross_validation import cross_val_predict
#http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.cross_val_predict.html#sklearn.cross_validation.cross_val_predict

#Linear Discriminants. 

ldisc = LinearDiscriminantAnalysis(priors=None)
ldisc.fit(train_vect,train_result)

ldisc.predict()

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0)